# Capstone 'Battle of the Neighborhoods' Final Project
<hr></hr>

##### Description of the Problem

For my final project, I will be helping to locate an Afghan restaurant in Toronto, Canada. I noticed in the Foursquare data used earlier in this course that there was a category for Afghan restaurants, and upon further research from the <a href="https://www.thecanadianencyclopedia.ca/en/article/afghan-canadians">Canadian Encyclopedia</a> I discovered that many of Canada's 84,000 Afghans live in the Toronto area. As I did not, upon cursory notice, discover a plethora of Afghan restaurants in Toronto, I wondered if the area would be a good market for such a restaurant. Minority ethnic groups, especially recent migrants, often due not have access to the cuisines of their ancestral lands outside of their homes. Restaurants provide a cultural connection to food and foster a sense of community within a neighborhood. As such, this project will try to focus on finding the best area to open an Afghan restaurant for the growing Afghan-Canadian population. The target audience for this would be potential investors or restauranteurs who are looking to open a restaurant catering to the Afghan-Canadian community.

##### Research Question

The question I will be exploring is: <i>Is Toronto a good market for an Afghan restaurant? What neighborhood would best support such a restaurant?</i>

##### Data to be used in Collection and Analysis

To do this I will use data from the following sources:
<ul>
    <li><b>Foursquare</b> - to get location data for the neighborhoods and existing Afghan restaurants. Foursquare provides data through an API, which I will make calls to in order to get data on the numbers and types of venues in certain neighborhoods, as well as their location data. I can use this data to geolocate Afghan restaurants in Toronto in order to see if there are any areas that are saturated with them, or lacking them. This data is both categorical and numeric in nature, though I will be focusing specifically on location data for geocoding (numeric, continuous), categorical data for the restaurant (e.g. 'Afghan Restaurant'), and numeric data on the frequency of these restaurants per neighborhood (numeric, continuous). </li>
    <li><a href="https://open.toronto.ca/dataset/neighbourhood-profiles/"><b>Canadian Census Wealth and Demographic Data for Toronto</b></a> - to get data on wealth indicators for Toronto to see if there is enough capital to support a business. As a wealth indicator I will be using income deciles to demonstrate areas where there is enough social capital to develop a restaurant. This data is provided online in machine readable formats, so I will probably use it as a CSV and enter it into pandas. This also will be used to get information on where Afghans might be living in the Toronto area so as to figure out where a potential market for the business might be. This data provides numbers on different ethnic groups in the Toronto area, so I will be able to utilize it to figure out what areas Afghans are living in. If I find a specific neighborhood, joined to this data, that has a high number of Afghan-Canadians then it would be considered a prime market for this restaurant. This data is numeric in form, though the key is categorical in nature (e.g. ethnicity would be 'Afghan'). I can use the numeric, continuous data, to determine the concentration of Afghan-Canadians in neighborhoods of Toronto.</li>
    <li><a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"><b> Toronto Neighborhood Location Data</b></a> - to get information on the neighborhoods to be used for clustering and visualizations. This data contains neighborhood names and postal codes which will be used in conjunction with the Foursquare data to locate restaurants.</li>
</ul>

This data will enable me to provide market analysis to find out if there is a neighborhood in Toronto that would be best for an Afghan restaurant. If there is not, this analysis will warn against such a decision.
<hr></hr>

## Part 1: Data Collection and Analysis

### Step 1: Scrape the Wikipedia Page for Toronto Neighborhood Location Data

In [2]:
import pandas as pd
import requests

In [3]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 5.3MB/s ta 0:00:011     |█████████████████████████▉      | 81kB 18.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup

In [5]:
BeautifulSoup

bs4.BeautifulSoup

In [6]:
r1 = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r1.content,'html.parser')
table = soup.find(lambda tag: tag.name =='table' and ("wikitable" in tag['class']))

In [7]:
df = pd.read_html(str(table), flavor='bs4')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step 2: Remove the "Not Assigned" data
Do this by creating a dataframe where it selects those that are not assigned, then inverse it.

In [8]:
df_2 = (df['Borough'] == 'Not assigned')|(df['Neighbourhood'] == 'Not assigned')
df = df[~df_2]

### Step 3: Check to see if there are any "Not Assigned" variables left

In [9]:
(df.Borough == 'Not assigned').sum()

0

In [10]:
(df.Neighbourhood == 'Not Assigned').sum()

0

In [11]:
post=df.Postcode.unique()

### Step 4: Group the dataframe by postcode
Some entries have the same postcode, so we need to group them together based on the unique postcode using the variable above.

In [12]:
Toronto = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
for code in post:
    temp_df = df[['Borough','Neighbourhood']][df['Postcode'] == code]
    boro = temp_df.Borough.unique()
    hood = temp_df.Neighbourhood.unique()
    Toronto = Toronto.append({
        'Postcode':code,
        'Borough':",".join(boro),
        'Neighbourhood':",".join(hood)},ignore_index=True)

In [13]:
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M9A,Etobicoke,Islington Avenue


In [14]:
Toronto.shape

(102, 3)

### Step 5: Geocode the Post Codes
#### Note: I got this data from the CSV used in the week 3 assignment.

In [15]:
postalcodes_from_csv = pd.read_csv('http://cocl.us/Geospatial_data')

In [16]:
postalcodes_from_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
Toronto = Toronto.sort_values(by=['Postcode'])
Toronto.reset_index(inplace=True, drop=True)

In [18]:
pcode = postalcodes_from_csv.sort_values(by=['Postal Code'])

In [19]:
pcode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
Toronto=pd.concat([Toronto, pcode[['Latitude','Longitude']]], axis = 1)

In [21]:
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Step 6: Find only boroughs that are in Toronto

In [22]:
Toronto = Toronto[Toronto.Borough.str.contains('Toronto', na=False)]

In [23]:
Toronto.reset_index( inplace = True)
Toronto

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [24]:
Toronto.drop(['index'], axis = 1, inplace = True)

##### Then I reset the index so it would be properly ordered, starting at 0.

In [25]:
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


### Step 7: Mapmaking
###### For the next part, I will map the above boroughs.

In [26]:
import folium

In [27]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

### Step 8: Get Foursquare Data
##### Next, we will get information from Foursquare's API regarding establishments in each borough.

In [28]:
# @hidden_cell
CLIENT_ID = 'RNKIHBWGNRSAOCCLS1HI3CCQYEZTMJ5NF4P054EJFN5JLWKI' # your Foursquare ID
CLIENT_SECRET = 'PAJ2BMGBQEVPBU21HNV4IO3UGUQKLI4GEYNUTRDUW4WRH4NO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [29]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json

def getNearbyVenues(postcodes, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    postcodes_done = ""
    for code, lat, lng in zip(postcodes, latitudes, longitudes):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        res = requests.get(url).json()["response"]
        if 'groups' not in res:
            continue;
        
        postcodes_done += code+","
        results = res['groups'][0]['items']
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print(postcodes_done[:-1])
    return(nearby_venues)

Earlier, we created a dataframe named 'Toronto' that only selected boroughs in Toronto. Now, we will combine that with the above pull request into a new dataframe so we have venues in these areas.

In [30]:
toronto_venues = getNearbyVenues(Toronto['Postcode'], Toronto['Latitude'], Toronto['Longitude'],radius=500, limit=100)

M4E,M4K,M4L,M4M,M4N,M4P,M4R,M4S,M4T,M4V,M4W,M4X,M4Y,M5A,M5B,M5C,M5E,M5G,M5H,M5J,M5K,M5L,M5N,M5P,M5R,M5S,M5T,M5V,M5W,M5X,M6G,M6H,M6J,M6K,M6P,M6R,M6S,M7Y


In [31]:
toronto_venues.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


### Step 9: One-Hot Encoding
Now, we will do one-hot encoding so we can make the categorical data ready to be read by a machine learning algorithm.

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(1718, 237)

In [33]:
toronto_grouped = toronto_onehot.groupby('Postcode').sum()

In [34]:
toronto_grouped.reset_index(inplace=True)
toronto_grouped.head()

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4K,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,M4L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4M,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,1
4,M4N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Take note of the 'Afghan Restaurant Category' !

### Step 10: Locate the Afghan Restaurants in Toronto

In [35]:
df2 = toronto_grouped[['Postcode','Afghan Restaurant']]

In [36]:
df2

,Postcode,Afghan Restaurant
0,M4E,0
1,M4K,0
2,M4L,0
3,M4M,0
4,M4N,0
5,M4P,0
6,M4R,0
7,M4S,0
8,M4T,0
9,M4V,0


Looking at the dataframe, the only Afghan Restaurant in Toronto is in the M4Y postcode. Looking at the dataframe we made in Step 6, we can see that this is in Downtown Toronto in the Church and Wellesley neighborhood. In my mind, this means that Toronto is not saturated with Afghan restaurants, so we might be able to place this anywhere! However, we need to make sure that the community is there and has the money to support the business, which is where the Canadian Census Data comes in.

### Step 11: Import Canadian Census Data for Toronto Neighborhoods
I will make this data available on my Github page as a CSV -- it was downloaded from the hyperlink in the above data section and cleaned by deleting irrelevant data.

In [37]:
neighborhoods_id = pd.read_csv('Toronto_Neighborhoods.csv')

In [43]:
neighborhoods_id.head(14)

,Topic,Characteristic,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Income of economic families in 2015,Total - Economic family income decile group fo...,"28,825","23,475","12,030","28,650","27,005","15,580","25,600","21,135",...,"16,725","22,150","53,005","12,430","7,850","13,200","11,810","12,370","27,570","14,020"
1,Income of economic families in 2015,In the bottom half of the distribution,"18,535","15,205","4,735","12,080","10,760","8,150","15,230","10,745",...,"8,545","10,895","36,245","5,025","3,490","6,355","3,840","4,165","19,105","7,735"
2,Income of economic families in 2015,In the bottom decile,"4,195","3,905",725,"4,810","2,600","1,825","8,820","4,005",...,"3,620","2,005","9,965","1,490",890,"1,685","1,185","1,340","5,410","1,630"
3,Income of economic families in 2015,In the second decile,"5,030","3,565",815,"2,225","2,195","1,865","2,020","1,995",...,"1,740","2,450","8,715","1,040",715,"1,480",660,675,"4,135","1,665"
4,Income of economic families in 2015,In the third decile,"3,700","2,940","1,005","1,710","2,050","1,595","1,660","1,585",...,"1,145","2,295","6,815",850,625,"1,175",630,715,"3,625","1,775"
5,Income of economic families in 2015,In the fourth decile,"2,990","2,590","1,060","1,630","1,875","1,455","1,380","1,550",...,965,"2,120","5,845",780,610,965,665,715,"3,235","1,380"
6,Income of economic families in 2015,In the fifth decile,"2,610","2,210","1,140","1,735","2,045","1,410","1,350","1,620",...,"1,090","2,030","4,955",870,645,"1,060",700,730,"2,710","1,290"
7,Income of economic families in 2015,In the top half of the distribution,"10,280","8,265","7,290","16,570","16,245","7,435","10,365","10,395",...,"8,175","11,255","16,750","7,405","4,355","6,840","7,965","8,210","8,460","6,290"
8,Income of economic families in 2015,In the sixth decile,"2,500","2,015","1,265","1,720","2,105","1,285","1,400","1,475",...,"1,110","2,145","4,520","1,030",735,"1,030",815,780,"2,470","1,285"
9,Income of economic families in 2015,In the seventh decile,"2,340","1,840","1,420","1,820","2,230","1,240","1,470","1,670",...,"1,240","2,080","4,240",985,785,"1,085",850,880,"2,095","1,260"


Above, you can see the data for Toronto neighborhoods -- this includes the ethnic origin of the population ('Afghan' being the operative population) as well as income statistics for the neighborhoods.

In [53]:
neighborhoods_id.transpose()
#I wanted to transpose the data so I can find use the pandas.sort_values() function to see where most of the Afghan population of Toronto is living. 

,1,2,3,4,5,6,7,8,9,10,11,12,13
Topic,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Ethnic origin population
Characteristic,In the bottom half of the distribution,In the bottom decile,In the second decile,In the third decile,In the fourth decile,In the fifth decile,In the top half of the distribution,In the sixth decile,In the seventh decile,In the eighth decile,In the ninth decile,In the top decile,Afghan
Agincourt North,"18,535","4,195","5,030","3,700","2,990","2,610","10,280","2,500","2,340","2,260","1,900","1,305",85
Agincourt South-Malvern West,"15,205","3,905","3,565","2,940","2,590","2,210","8,265","2,015","1,840","1,800","1,505","1,125",255
Alderwood,"4,735",725,815,"1,005","1,060","1,140","7,290","1,265","1,420","1,500","1,750","1,345",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wychwood,"6,355","1,685","1,480","1,175",965,"1,060","6,840","1,030","1,085","1,235","1,410","2,080",0
Yonge-Eglinton,"3,840","1,185",660,630,665,700,"7,965",815,850,"1,060","1,480","3,770",10
Yonge-St.Clair,"4,165","1,340",675,715,715,730,"8,210",780,880,"1,030","1,300","4,220",35
York University Heights,"19,105","5,410","4,135","3,625","3,235","2,710","8,460","2,470","2,095","1,830","1,380",700,340


In [65]:
neighborhoods_id = neighborhoods_id.transpose()

In [66]:
neighborhoods_id.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
Topic,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Income of economic families in 2015,Ethnic origin population
Characteristic,In the bottom half of the distribution,In the bottom decile,In the second decile,In the third decile,In the fourth decile,In the fifth decile,In the top half of the distribution,In the sixth decile,In the seventh decile,In the eighth decile,In the ninth decile,In the top decile,Afghan
Agincourt North,"18,535","4,195","5,030","3,700","2,990","2,610","10,280","2,500","2,340","2,260","1,900","1,305",85
Agincourt South-Malvern West,"15,205","3,905","3,565","2,940","2,590","2,210","8,265","2,015","1,840","1,800","1,505","1,125",255
Alderwood,"4,735",725,815,"1,005","1,060","1,140","7,290","1,265","1,420","1,500","1,750","1,345",10


In [76]:
neighborhoods_id.drop(neighborhoods_id.index[0], inplace = True)
# I had to drop the index as it had redundant categorical data.

In [72]:
neighborhoods_id.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
Characteristic,In the bottom half of the distribution,In the bottom decile,In the second decile,In the third decile,In the fourth decile,In the fifth decile,In the top half of the distribution,In the sixth decile,In the seventh decile,In the eighth decile,In the ninth decile,In the top decile,Afghan
Agincourt North,"18,535","4,195","5,030","3,700","2,990","2,610","10,280","2,500","2,340","2,260","1,900","1,305",85
Agincourt South-Malvern West,"15,205","3,905","3,565","2,940","2,590","2,210","8,265","2,015","1,840","1,800","1,505","1,125",255
Alderwood,"4,735",725,815,"1,005","1,060","1,140","7,290","1,265","1,420","1,500","1,750","1,345",10
Annex,"12,080","4,810","2,225","1,710","1,630","1,735","16,570","1,720","1,820","2,210","2,830","7,995",35


### Step 12: Locate the best Neighborhood for the restaurant
To do this, I will look at the decile economic data as well as the population of Afghan-Canadians in the dataset.

In [75]:
neighborhoods_id.sort_values(by=[13])

,1,2,3,4,5,6,7,8,9,10,11,12,13
Characteristic,In the bottom half of the distribution,In the bottom decile,In the second decile,In the third decile,In the fourth decile,In the fifth decile,In the top half of the distribution,In the sixth decile,In the seventh decile,In the eighth decile,In the ninth decile,In the top decile,Afghan
Lawrence Park North,"2,780",770,485,415,510,605,"11,775",705,890,"1,195","2,040","6,955",0
Leaside-Bennington,"3,385",885,630,610,595,655,"13,265",845,"1,080","1,335","2,035","7,965",0
Long Branch,"5,265","1,340","1,070",965,920,970,"4,790",870,900,875,"1,060","1,095",0
Maple Leaf,"5,910",965,"1,310","1,375","1,185","1,070","3,945",895,895,705,750,710,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
South Parkdale,"14,680","4,720","3,470","2,565","2,115","1,815","5,965","1,420","1,365","1,165","1,040",995,90
Willowdale East,"28,045","12,600","4,985","3,855","3,285","3,315","22,210","3,365","3,620","3,840","4,990","6,425",95
Newtonbrook West,"15,320","4,955","3,305","2,765","2,315","1,985","8,305","1,900","1,675","1,480","1,540","1,715",95
Humber Summit,"8,210","1,600","1,845","1,830","1,590","1,345","4,195","1,185","1,040",930,680,355,95


As we can see above, there are 4 neighborhoods that each have 95 Afghan persons within them: Willowdale-East, Newtonbrook-West, Humber Summit, and Willowridge-Martingrove-Richview. As I want to focus this research on finding a market for an Afghan restaurant, I will keep these neighborhoods in mind. However, I also would like to see which of these neighborhoods is wealthiest. To do so, I will compare the numbers in the top 5 deciles to the lowest 5 deciles. Willowdale East has more in the bottom half than top half, as does Newtonbrook West and Humber Summit. By income, Willowridge-Martingrove-Richview has the inverse, with more households in the top half of the distribution. As this is miles away from the Afghan restaurant downtown, I feel this would be the neighborhood to base an Afghan restaurant in. Based on this analysis, I would select this neighborhood to host the restaurant. However, as this is mostly a residential neighborhood, and there are only 95 